In [1]:
%pip install ipywidgets -q
%pip install prophet -q

In [2]:
import pandas as pd
from prophet import Prophet

In [3]:
cash_flow_components = pd.read_csv("./assets/brazilian_companies_most_relevant_attr_for_free_cash_flow.csv", header=[0, 1], index_col=0)
cash_flow_components

## Pré-processamento dos dados

In [4]:
from utils import get_columns_from_header_0

tickers = get_columns_from_header_0(cash_flow_components)
tickers

In [5]:
from typing import List
from pandas import DataFrame


def to_time_series(ticker_df: DataFrame) -> List[DataFrame]:
  time_series = []

  for column in ticker_df.columns:
    column_df = DataFrame(ticker_df[column])
    column_df.insert(loc=0, column="ds", value=pd.to_datetime(column_df.index))
    column_df.rename(columns={column: "y"}, inplace=True)
    column_df.reset_index(drop=True, inplace=True)
    column_df = column_df.sort_values(by="ds")

    time_series.append(column_df)

  return time_series


## Previsão dos componentes do fluxo de caixa

In [6]:
def predict(ticker_df: DataFrame, n_quarters: int = 1) -> DataFrame:
    ticker_time_series = to_time_series(ticker_df)

    predictions = pd.DataFrame()

    for i, dataset in enumerate(ticker_time_series):
        if len(dataset) < 2:
            continue

        model = Prophet()
        model.fit(dataset)

        # Prever 1 ano de um dos componentes do fluxo de caixa
        future = model.make_future_dataframe(periods=n_quarters, freq="BQ")
        forecast = model.predict(future)

        forecast = forecast[["ds", "yhat"]].rename(
            columns={"yhat": ticker_df.columns[i]})
        forecast.sort_values(by="ds")
        forecast.set_index("ds", inplace=True)
        forecast.index.name = None
        forecast = forecast.transpose()

        if predictions.empty:
            predictions = forecast
            continue

        predictions = pd.concat([predictions, forecast])

    return predictions

In [7]:
def create_predicted_cash_flow_components(n_quarters: int):
    predicted_cash_flow_components = DataFrame()

    for ticker in tickers:
        ticker_df = cash_flow_components[ticker].transpose()
        predictions = predict(ticker_df, n_quarters)

        predictions.columns = pd.MultiIndex.from_product(
            [[ticker], predictions.columns])

        if predicted_cash_flow_components.empty:
            predicted_cash_flow_components = predictions
            continue

        predicted_cash_flow_components = pd.concat(
            [predicted_cash_flow_components, predictions], axis=1)
        
    predicted_cash_flow_components.to_csv("./assets/predicted_cash_flow_components.csv")

    return predicted_cash_flow_components

In [8]:
create_predicted_cash_flow_components(4 * 2)